In [ ]:
!pip install boto3 langchain pypdf unstructured[pdf] langchain-community

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import json
import boto3

In [ ]:
boto3_bedrock = boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='{}',aws_secret_access_key='{}')

In [ ]:
def summarizer(prompt_data):
  inputText=prompt_data
  body_part=json.dumps({'inputText': inputText,
  'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
  response = boto3_bedrock.invoke_model(
  body=body_part,
  contentType="application/json",
  accept="application/json",
  modelId='amazon.titan-text-express-v1'
  )
  output_text=json.loads(response['body'].read())['results'][0]['outputText']
  return output_text

In [ ]:
def read_pdf_and_split(filename):
  loader = UnstructuredPDFLoader(filename)
  data = loader.load()
  print(data)
  splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=100,
  length_function=len,
  add_start_index=True
  )
  splitted_text = splitter.split_documents(data)

  return splitted_text


In [ ]:
pdf_document = read_pdf_and_split('/content/YOGI_2_0.pdf')

In [ ]:
pdf_document

In [ ]:
len(pdf_document)

In [ ]:
summary= ""
for i in pdf_document:
  # gathering the text content of that specific chunk
  chunk_content = i.page_content
  # creating the prompt that will be passed into Bedrock with the text content of the chunk
  prompt = f"""Human: Provide a detailed summary for the chunk of text provided to you:
  Text: {chunk_content}"""
  # passing the prompt into the summarizer function to generate the summary of that chunk, and appending it to
  # the summary string
  summary += summarizer(prompt)

final_summary_prompt = f"""Human: You will be given a set of summaries from a document. Create a cohesive
summary from the provided individual summaries. The summary should very detailed.
Summaries: {summary}"""
# generating the final summary of all the summaries we have previously generated.
print(summarizer(final_summary_prompt))